# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Project Setup

In [66]:
# conda install geopandas

Retrieving notices: ...working... done
Channels:
 - defaults
 - gurobi
Platform: osx-64
Solving environment: done

## Package Plan ##

  environment location: /Users/shaoziheng/anaconda3

  added / updated specs:
    - geopandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.6.0               |  py311hecd8cb5_0          51 KB
    ca-certificates-2024.11.26 |       hecd8cb5_0         132 KB
    cairo-1.16.0               |       h3ce6f7e_5        1013 KB
    click-plugins-1.1.1        |     pyhd3eb1b0_0          10 KB
    cligj-0.7.2                |     pyhd3eb1b0_0          11 KB
    conda-24.11.0              |  py311hecd8cb5_0         1.2 MB
    expat-2.6.3                |       h6d0c2b6_0         151 KB
    fiona-1.9.5                |  py311hdb55bb0_0         914 KB
    folium-0.14.0              |  py311hecd8cb5_0         128 KB
    fontconfig-2.14.1          |       h

 ... (more hidden) ...

pandas-2.1.4         | 14.4 MB   | #                                     |   3% 


libspatialite-5.1.0  | 3.9 MB    | ####1                                 |  11% 



proj-9.3.1           | 2.9 MB    | #####6                                |  15% 
libgdal-3.6.2        | 9.0 MB    | 5                                     |   1% 



proj-9.3.1           | 2.9 MB    | ##############5                       |  39% 


pandas-2.1.4         | 14.4 MB   | ###6                                  |  10% 

hdf5-1.12.1          | 5.0 MB    | ####1                                 |  11% 
libgdal-3.6.2        | 9.0 MB    | #######                               |  19% 



pandas-2.1.4         | 14.4 MB   | ######7                               |  18% 

hdf5-1.12.1          | 5.0 MB    | ###########6                          |  31% 
libgdal-3.6.2        | 9.0 MB    | ##########9                           |  30% 


libspatialite-5.1.0  | 3.9 MB    | ################################ 

libxml2-2.10.4       | 666 KB    | ##################################### | 100% 




















shapely-2.0.5        | 565 KB    | ##################################### | 100% 




















shapely-2.0.5        | 565 KB    | ##################################### | 100% 

















geos-3.8.0           | 815 KB    | ##################################### | 100% 

















geos-3.8.0           | 815 KB    | ##################################### | 100% 






















 ... (more hidden) ...






















 ... (more hidden) ...





















pyproj-3.6.1         | 518 KB    | ##################################### | 100% 





















pandas-2.1.4         | 14.4 MB   | ##################################### | 100% 






















                                                                                
                                                                                

                                          

In [1]:
# all import statements needed for the project, for example:

import os

import bs4
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db
import re
import geopandas as gpd
from math import ceil

In [2]:
# any constants you might need; some have been added for you, and 
# some you need to fill in

TLC_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

TAXI_ZONES_DIR = "/Users/shaoziheng/Desktop/4501/project/datasets/taxi_zones"
TAXI_ZONES_SHAPEFILE = f"{TAXI_ZONES_DIR}/taxi_zones.shp"
WEATHER_CSV_DIR = "/Users/shaoziheng/Desktop/4501/project/datasets/weather"

CRS = 4326  # coordinate reference system

# (lat, lon)
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
LGA_BOX_COORDS = ((40.763589, -73.891745), (40.778865, -73.854838))
JFK_BOX_COORDS = ((40.639263, -73.795642), (40.651376, -73.766264))
EWR_BOX_COORDS = ((40.686794, -74.194028), (40.699680, -74.165205))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [3]:
# Make sure the QUERY_DIRECTORY exists
try:
    os.mkdir(QUERY_DIRECTORY)
except Exception as e:
    if e.errno == 17:
        # the directory already exists
        pass
    else:
        raise

## Part 1: Data Preprocessing

### Load Taxi Zones

In [4]:
def load_taxi_zones(shapefile):
    g = gpd.read_file(shapefile)
    g = g.to_crs(4326)
    g['longitude'] = g.centroid.x
    g['latitude'] = g.centroid.y
    return g

In [8]:
taxi_zones=load_taxi_zones('taxi_zones.shp')

/var/folders/91/5yc0y4vn7v16zydty26b24t80000gn/T/ipykernel_96268/2285928404.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  g['longitude'] = g.centroid.x
/var/folders/91/5yc0y4vn7v16zydty26b24t80000gn/T/ipykernel_96268/2285928404.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  g['latitude'] = g.centroid.y


In [9]:
taxi_zones

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry,longitude,latitude
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((-74.18445 40.69500, -74.18449 40.695...",-74.174000,40.691831
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ...",-73.831299,40.616745
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((-73.84793 40.87134, -73.84725 40.870...",-73.847422,40.864474
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((-73.97177 40.72582, -73.97179 40.725...",-73.976968,40.723752
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((-74.17422 40.56257, -74.17349 40.562...",-74.188484,40.552659
...,...,...,...,...,...,...,...,...,...
258,259,0.126750,0.000395,Woodlawn/Wakefield,259,Bronx,"POLYGON ((-73.85107 40.91037, -73.85207 40.909...",-73.852215,40.897932
259,260,0.133514,0.000422,Woodside,260,Queens,"POLYGON ((-73.90175 40.76078, -73.90147 40.759...",-73.906306,40.744235
260,261,0.027120,0.000034,World Trade Center,261,Manhattan,"POLYGON ((-74.01333 40.70503, -74.01327 40.704...",-74.013023,40.709139
261,262,0.049064,0.000122,Yorkville East,262,Manhattan,"MULTIPOLYGON (((-73.94383 40.78286, -73.94376 ...",-73.946510,40.775932


In [10]:
pd.read_parquet('yellow_tripdata_2024-01.parquet')

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.70,1.00,0.5,0.00,0.00,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.00,3.50,0.5,3.75,0.00,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.30,3.50,0.5,3.00,0.00,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.00,3.50,0.5,2.00,0.00,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.90,3.50,0.5,3.20,0.00,1.0,16.10,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964619,2,2024-01-31 23:45:59,2024-01-31 23:54:36,NaN,3.18,NaN,None,107,263,0,15.77,0.00,0.5,2.00,0.00,1.0,21.77,NaN,NaN
2964620,1,2024-01-31 23:13:07,2024-01-31 23:27:52,NaN,4.00,NaN,None,114,236,0,18.40,1.00,0.5,2.34,0.00,1.0,25.74,NaN,NaN
2964621,2,2024-01-31 23:19:00,2024-01-31 23:38:00,NaN,3.33,NaN,None,211,25,0,19.97,0.00,0.5,0.00,0.00,1.0,23.97,NaN,NaN
2964622,2,2024-01-31 23:07:23,2024-01-31 23:25:14,NaN,3.06,NaN,None,107,13,0,23.88,0.00,0.5,5.58,0.00,1.0,33.46,NaN,NaN


In [ ]:
# def lookup_coords_for_taxi_zone_id(zone_loc_id, loaded_taxi_zones):
#     g['longitude'] = g.centroid.x
#     g['latitude'] = g.centroid.y
#     return 

### Calculate Sample Size

In [ ]:
def calculate_sample_size(population, confidence_level=0.95, margin_of_error=0.05):
    Z = {0.9: 1.645, 0.95: 1.96, 0.99: 2.576}[confidence_level]
    p = 0.5
    e = margin_of_error
    sample_size = (Z**2 * p * (1 - p)) / e**2
    if population < 1e6:  # Finite population correction
        sample_size = sample_size / (1 + (sample_size - 1) / population)
    return ceil(sample_size)

### Common Functions

In [6]:
def get_all_urls_from_tlc_page(taxi_page):
    response = requests.get(taxi_page)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch page content: {response.status_code}")
    soup = BeautifulSoup(response.text, "html.parser")
    return soup

In [14]:
def filter_parquet_urls(soup):
    yellow_taxi_links = []
    yellow_links = soup.find_all('a', {'href': re.compile(r"yellow_tripdata_202[0-3]-\d{2}\.parquet|yellow_tripdata_2024-(0[1-8])\.parquet")})
    for link in yellow_links:
        yellow_taxi_links.append(link['href'])
    
    hvfhv_links = []
    hvfhv_links_soup = soup.find_all('a', {'href': re.compile(r"fhvhv_tripdata_202[0-3]-\d{2}\.parquet|fhvhv_tripdata_2024-(0[1-8])\.parquet")})
    for link in hvfhv_links_soup:
        hvfhv_links.append(link['href'])
    
    return yellow_taxi_links, hvfhv_links

In [19]:
soup=get_all_urls_from_tlc_page(TLC_URL)
yellow_taxi_links, hvfhv_links = filter_parquet_urls(soup)

### Process Taxi Data

In [20]:
def get_and_clean_taxi_month(url):
    
    taxi_zones = taxi_zones[['LocationID', 'longitude', 'latitude', 'zone', 'borough']]
    lat_map = dict(zip(g['LocationID'], g['latitude']))
    lon_map = dict(zip(g['LocationID'], g['longitude']))
    
    df=pd.read_parquet(url)
    
    # map Pickup location ID and Dropoff location ID to corresponding lat and log
    # and create new columns for these latitude and longitude
    df['pickup_latitude']  = df['PULocationID'].map(lat_map)
    df['pickup_longitude'] = df['PULocationID'].map(lon_map)
    df['dropoff_latitude'] = df['DOLocationID'].map(lat_map)
    df['dropoff_longitude']= df['DOLocationID'].map(lon_map)
    
    # removing unnecessary columns
    
    
    # removing invalid data points
    

    # normalizing column names
    
    
    # removing trips that start and/or end outside of the NEW_YORK_BOX
    df = df[df['pickup_latitude'] >= NEW_YORK_BOX_COORDS[0][0]]
    df = df[df['pickup_latitude'] <= NEW_YORK_BOX_COORDS[1][0]]
    df = df[df['pickup_longitude'] >= NEW_YORK_BOX_COORDS[0][1]]
    df = df[df['pickup_longitude'] <= NEW_YORK_BOX_COORDS[1][1]]
        
    df = df[df['dropoff_latitude'] >= NEW_YORK_BOX_COORDS[0][0]]
    df = df[df['dropoff_latitude'] <= NEW_YORK_BOX_COORDS[1][0]]
    df = df[df['dropoff_longitude'] >= NEW_YORK_BOX_COORDS[0][1]]
    df = df[df['dropoff_longitude'] <= NEW_YORK_BOX_COORDS[1][1]]
    
    

In [21]:
def get_and_clean_taxi_data(parquet_urls):
    all_taxi_dataframes = []
    
    for url in yellow_taxi_links:
        file = url.split('/')[-1]
        if not os.path.exists(file):
            file = requests.get(url)
            with open(file, "wb") as f:
                f.write(file.content)
            
        # read local files and clean
        dataframe = get_and_clean_taxi_month(file)  
        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.contact(all_taxi_dataframes)
    return taxi_data

In [22]:
def get_taxi_data():
    all_urls = get_all_urls_from_taxi_page(TLC_URL)
    all_parquet_urls = find_taxi_parquet_urls(all_urls)
    taxi_data = get_and_clean_taxi_data(all_parquet_urls)
    return taxi_data

In [23]:
taxi_data = get_taxi_data()

NameError: name 'get_all_urls_from_taxi_page' is not defined

In [53]:
taxi_data.head()

NameError: name 'taxi_data' is not defined

In [ ]:
taxi_data.info()

In [ ]:
taxi_data.describe()

### Processing Uber Data

In [ ]:
def get_and_clean_uber_month(url):
    raise NotImplementedError()

In [ ]:
def get_and_clean_uber_data(parquet_urls):
    all_uber_dataframes = []
    
    for parquet_url in parquet_urls:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_uber_month(parquet_url)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        
        all_uber_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    uber_data = pd.contact(all_uber_dataframes)
    return uber_data

In [ ]:
def load_and_clean_uber_data():
    raise NotImplementedError()

In [ ]:
def get_uber_data():
    all_urls = get_all_urls_from_tlc_page(TLC_URL)
    all_parquet_urls = find_parquet_urls(all_urls)
    taxi_data = get_and_clean_uber_data(all_parquet_urls)
    return taxi_data

In [ ]:
uber_data = get_uber_data()

In [ ]:
uber_data.head()

In [ ]:
uber_data.info()

In [ ]:
uber_data.describe()

### Processing Weather Data

In [ ]:
def get_all_weather_csvs(directory):
    raise NotImplementedError()

In [ ]:
def clean_month_weather_data_hourly(csv_file):
    raise NotImplementedError()

In [ ]:
def clean_month_weather_data_daily(csv_file):
    raise NotImplementedError()

In [ ]:
def load_and_clean_weather_data():
    weather_csv_files = get_all_weather_csvs(WEATHER_CSV_DIR)
    
    hourly_dataframes = []
    daily_dataframes = []
        
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

In [ ]:
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

In [ ]:
hourly_weather_data.head()

In [ ]:
hourly_weather_data.info()

In [ ]:
hourly_weather_data.describe()

In [ ]:
daily_weather_data.head()

In [ ]:
daily_weather_data.info()

In [ ]:
daily_weather_data.describe()

## Part 2: Storing Cleaned Data

In [ ]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
TODO
"""

DAILY_WEATHER_SCHEMA = """
TODO
"""

TAXI_TRIPS_SCHEMA = """
TODO
"""

UBER_TRIPS_SCHEMA = """
TODO
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [ ]:
# create the tables with the schema files
with engine.connect() as connection:
    pass

### Add Data to Database

In [ ]:
def write_dataframes_to_table(table_to_df_dict):
    raise NotImplemented()

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

### Query 1

In [ ]:
QUERY_1_FILENAME = ""

QUERY_1 = """
TODO
"""

In [ ]:
# execute query either via sqlalchemy
with engine.connect() as con:
    results = con.execute(db.text(QUERY_1)).fetchall()
results

# or via pandas
pd.read_sql(QUERY_1, con=engine)

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)